In [ ]:
using LowLevelFEM, LinearAlgebra, SparseArrays

In [ ]:
function poissonMatrixVector(
    problem::Problem;
    coefficient::TensorField)

    @assert problem.pdim == problem.dim
    gmsh.model.setCurrent(problem.name)

    dim = problem.dim
    pdim = problem.pdim
    non = problem.non
    dof = pdim * non

    # elementwise tensor field (node values gathered per element)
    S = nodesToElements(coefficient)
    Se = Dict(zip(S.numElem, S.A))

    lengthOfIJV = LowLevelFEM.estimateLengthOfIJV(problem)
    I = Vector{Int}(undef, lengthOfIJV)
    J = Vector{Int}(undef, lengthOfIJV)
    V = Vector{Float64}(undef, lengthOfIJV)
    pos = 1

    for mat in problem.material
        for (edim, etag) in gmsh.model.getEntitiesForPhysicalName(mat.phName)

            elemTypes, elemTags, elemNodeTags =
                gmsh.model.mesh.getElements(edim, etag)

            for it in eachindex(elemTypes)
                et = elemTypes[it]
                _, _, order, numNodes, _, _ =
                    gmsh.model.mesh.getElementProperties(et)

                intPoints, intWeights =
                    gmsh.model.mesh.getIntegrationPoints(et, "Gauss" * string(2order + 1))

                # --- get Lagrange shape functions for proper interpolation
                _, hfun, _ =
                    gmsh.model.mesh.getBasisFunctions(et, intPoints, "Lagrange")
                H = reshape(hfun, numNodes, :)

                # --- gradients
                _, dfun, _ =
                    gmsh.model.mesh.getBasisFunctions(et, intPoints, "GradLagrange")
                ∇h = reshape(dfun, :, length(intWeights))

                for (e, elem) in enumerate(elemTags[it])
                    nodeTags = elemNodeTags[it][(e-1)*numNodes+1:e*numNodes]

                    jac, jacDet, _ =
                        gmsh.model.mesh.getJacobian(elem, intPoints)
                    Jac = reshape(jac, 3, :)

                    Ke = zeros(pdim * numNodes, pdim * numNodes)

                    # tensor at element nodes (9×numNodes)
                    Snode = Se[elem]

                    for k in eachindex(intWeights)
                        invJ = inv(Jac[1:dim, 3k-2:3k])'
                        w = jacDet[k] * intWeights[k]

                        # --- interpolate S to Gauss point using shape functions
                        Sgp = zeros(dim, dim)
                        for a in 1:numNodes
                            Sa = reshape(Snode[9a-8:9a, 1], dim, dim)
                            Sgp .+= H[a, k] * Sa
                        end

                        for a in 1:numNodes, b in 1:numNodes
                            ∇Na = invJ * ∇h[3a-2:3a-(3-dim), k]
                            ∇Nb = invJ * ∇h[3b-2:3b-(3-dim), k]

                            for i in 1:dim, j in 1:dim
                                ia = (a - 1) * pdim + i
                                ib = (b - 1) * pdim + j
                                Ke[ia, ib] += ∇Na[i] * Sgp[i, j] * ∇Nb[j] * w
                            end
                        end
                    end

                    # scatter
                    for a in 1:pdim*numNodes, b in 1:pdim*numNodes
                        I[pos] = (nodeTags[div(a - 1, pdim)+1] - 1) * pdim + mod(a - 1, pdim) + 1
                        J[pos] = (nodeTags[div(b - 1, pdim)+1] - 1) * pdim + mod(b - 1, pdim) + 1
                        V[pos] = Ke[a, b]
                        pos += 1
                    end
                end
            end
        end
    end

    resize!(I, pos - 1)
    resize!(J, pos - 1)
    resize!(V, pos - 1)

    K = sparse(I, J, V, dof, dof)
    dropzeros!(K)
    return SystemMatrix(K, problem)
end


In [ ]:
function gradDivMatrixF(
    problem::Problem;
    coefficient::Union{Number,ScalarField},
    F::TensorField)

    @assert problem.pdim == problem.dim
    gmsh.model.setCurrent(problem.name)

    dim = problem.dim
    pdim = problem.pdim
    non = problem.non
    dof = pdim * non

    # --- elementwise nodal tensors for F
    Fe = nodesToElements(F)
    Fe_map = Dict(zip(Fe.numElem, Fe.A))

    # --- elementwise nodal scalars for λ (if needed)
    λ_is_scalarfield = !(coefficient isa Number)
    if λ_is_scalarfield
        λe = nodesToElements(coefficient)
        λmap = Dict(zip(λe.numElem, λe.A))   # each entry: (numNodes×1) vector per element
    else
        λconst = Float64(coefficient)
    end

    lengthOfIJV = LowLevelFEM.estimateLengthOfIJV(problem)
    I = Vector{Int}(undef, lengthOfIJV)
    J = Vector{Int}(undef, lengthOfIJV)
    V = Vector{Float64}(undef, lengthOfIJV)
    pos = 1

    for mat in problem.material
        for (edim, etag) in gmsh.model.getEntitiesForPhysicalName(mat.phName)

            elemTypes, elemTags, elemNodeTags =
                gmsh.model.mesh.getElements(edim, etag)

            for it in eachindex(elemTypes)
                et = elemTypes[it]
                _, _, order, numNodes, _, _ =
                    gmsh.model.mesh.getElementProperties(et)

                intPoints, intWeights =
                    gmsh.model.mesh.getIntegrationPoints(et, "Gauss" * string(2order + 1))

                # --- Lagrange shape functions (for proper interpolation)
                _, hfun, _ =
                    gmsh.model.mesh.getBasisFunctions(et, intPoints, "Lagrange")
                H = reshape(hfun, numNodes, :)

                # --- GradLagrange
                _, dfun, _ =
                    gmsh.model.mesh.getBasisFunctions(et, intPoints, "GradLagrange")
                ∇h = reshape(dfun, :, length(intWeights))

                for (e, elem) in enumerate(elemTags[it])
                    nodeTags = elemNodeTags[it][(e-1)*numNodes+1:e*numNodes]

                    jac, jacDet, _ =
                        gmsh.model.mesh.getJacobian(elem, intPoints)
                    Jac = reshape(jac, 3, :)

                    Ke = zeros(pdim * numNodes, pdim * numNodes)

                    # nodal data for this element
                    Felem = Fe_map[elem]
                    if λ_is_scalarfield
                        λa = λmap[elem][:, 1]  # length = numNodes
                    end

                    for k in eachindex(intWeights)
                        invJ = inv(Jac[1:dim, 3k-2:3k])'
                        w = jacDet[k] * intWeights[k]

                        # --- interpolate F to GP using shape functions
                        Fgp = zeros(dim, dim)
                        for a in 1:numNodes
                            Na = H[a, k]
                            Fgp .+= Na * reshape(Felem[9a-8:9a, 1], dim, dim)
                        end

                        # --- interpolate λ to GP if needed
                        λgp = λ_is_scalarfield ? dot(λa, H[:, k]) : λconst

                        for a in 1:numNodes, b in 1:numNodes
                            ∇Na = invJ * ∇h[3a-2:3a-(3-dim), k]
                            ∇Nb = invJ * ∇h[3b-2:3b-(3-dim), k]

                            for i in 1:dim, j in 1:dim
                                ia = (a - 1) * pdim + i
                                ib = (b - 1) * pdim + j
                                Ke[ia, ib] += λgp * (Fgp[i, i] * ∇Na[i]) * (Fgp[j, j] * ∇Nb[j]) * w
                            end
                        end
                    end

                    # scatter
                    for a in 1:pdim*numNodes, b in 1:pdim*numNodes
                        I[pos] = (nodeTags[div(a - 1, pdim)+1] - 1) * pdim + mod(a - 1, pdim) + 1
                        J[pos] = (nodeTags[div(b - 1, pdim)+1] - 1) * pdim + mod(b - 1, pdim) + 1
                        V[pos] = Ke[a, b]
                        pos += 1
                    end
                end
            end
        end
    end

    resize!(I, pos - 1)
    resize!(J, pos - 1)
    resize!(V, pos - 1)

    K = sparse(I, J, V, dof, dof)
    dropzeros!(K)
    return SystemMatrix(K, problem)
end


In [ ]:
function poissonMatrixSymGradF(
    problem::Problem;
    coefficient::Union{Number,ScalarField},
    F::TensorField)

    @assert problem.pdim == problem.dim
    gmsh.model.setCurrent(problem.name)

    dim = problem.dim
    pdim = problem.pdim
    non = problem.non
    dof = pdim * non

    # --- elementwise nodal tensors for F
    Fe = nodesToElements(F)
    Fe_map = Dict(zip(Fe.numElem, Fe.A))

    # --- elementwise nodal scalars for μ (if needed)
    μ_is_scalarfield = !(coefficient isa Number)
    if μ_is_scalarfield
        μe = nodesToElements(coefficient)
        μmap = Dict(zip(μe.numElem, μe.A))  # each entry: (numNodes×1) vector per element
    else
        μconst = Float64(coefficient)
    end

    lengthOfIJV = LowLevelFEM.estimateLengthOfIJV(problem)
    I = Vector{Int}(undef, lengthOfIJV)
    J = Vector{Int}(undef, lengthOfIJV)
    V = Vector{Float64}(undef, lengthOfIJV)
    pos = 1

    for mat in problem.material
        for (edim, etag) in gmsh.model.getEntitiesForPhysicalName(mat.phName)

            elemTypes, elemTags, elemNodeTags =
                gmsh.model.mesh.getElements(edim, etag)

            for it in eachindex(elemTypes)
                et = elemTypes[it]
                _, _, order, numNodes, _, _ =
                    gmsh.model.mesh.getElementProperties(et)

                intPoints, intWeights =
                    gmsh.model.mesh.getIntegrationPoints(et, "Gauss" * string(2order + 1))

                # --- Lagrange shape functions (for proper interpolation of F and μ)
                _, hfun, _ =
                    gmsh.model.mesh.getBasisFunctions(et, intPoints, "Lagrange")
                H = reshape(hfun, numNodes, :)

                # --- GradLagrange
                _, dfun, _ =
                    gmsh.model.mesh.getBasisFunctions(et, intPoints, "GradLagrange")
                ∇h = reshape(dfun, :, length(intWeights))

                for (e, elem) in enumerate(elemTags[it])
                    nodeTags = elemNodeTags[it][(e-1)*numNodes+1:e*numNodes]

                    jac, jacDet, _ =
                        gmsh.model.mesh.getJacobian(elem, intPoints)
                    Jac = reshape(jac, 3, :)

                    Ke = zeros(pdim * numNodes, pdim * numNodes)

                    # nodal data for this element
                    Felem = Fe_map[elem]
                    if μ_is_scalarfield
                        μa = μmap[elem][:, 1]   # length = numNodes
                    end

                    for k in eachindex(intWeights)
                        invJ = inv(Jac[1:dim, 3k-2:3k])'
                        w = jacDet[k] * intWeights[k]

                        # --- interpolate F to Gauss point using shape functions
                        Fgp = zeros(dim, dim)
                        for a in 1:numNodes
                            Na = H[a, k]
                            Fgp .+= Na * reshape(Felem[9a-8:9a, 1], dim, dim)
                        end

                        # --- interpolate μ to Gauss point if needed
                        μgp = μ_is_scalarfield ? dot(μa, H[:, k]) : μconst

                        for a in 1:numNodes, b in 1:numNodes
                            ∇Na = invJ * ∇h[3a-2:3a-(3-dim), k]
                            ∇Nb = invJ * ∇h[3b-2:3b-(3-dim), k]

                            GradNa = zeros(dim, dim)
                            GradNb = zeros(dim, dim)
                            for i in 1:dim
                                GradNa[i, i] = ∇Na[i]
                                GradNb[i, i] = ∇Nb[i]
                            end

                            Ea = 0.5 * (Fgp' * GradNa + GradNa' * Fgp)
                            Eb = 0.5 * (Fgp' * GradNb + GradNb' * Fgp)

                            for i in 1:dim, j in 1:dim
                                ia = (a - 1) * pdim + i
                                ib = (b - 1) * pdim + j
                                Ke[ia, ib] += 2 * μgp * Ea[i, j] * Eb[i, j] * w
                            end
                        end
                    end

                    # scatter
                    for a in 1:pdim*numNodes, b in 1:pdim*numNodes
                        I[pos] = (nodeTags[div(a - 1, pdim)+1] - 1) * pdim + mod(a - 1, pdim) + 1
                        J[pos] = (nodeTags[div(b - 1, pdim)+1] - 1) * pdim + mod(b - 1, pdim) + 1
                        V[pos] = Ke[a, b]
                        pos += 1
                    end
                end
            end
        end
    end

    resize!(I, pos - 1)
    resize!(J, pos - 1)
    resize!(V, pos - 1)

    K = sparse(I, J, V, dof, dof)
    dropzeros!(K)
    return SystemMatrix(K, problem)
end


In [ ]:
function internalForceTL(problem::Problem, P::TensorField)

    gmsh.model.setCurrent(problem.name)

    dim = problem.dim
    pdim = problem.pdim
    non = problem.non
    dof = pdim * non

    Pe = nodesToElements(P)
    Pmap = Dict(zip(Pe.numElem, Pe.A))

    f = zeros(dof)

    for mat in problem.material
        for (edim, etag) in gmsh.model.getEntitiesForPhysicalName(mat.phName)

            elemTypes, elemTags, elemNodeTags =
                gmsh.model.mesh.getElements(edim, etag)

            for it in eachindex(elemTypes)
                et = elemTypes[it]
                _, _, order, numNodes, _, _ =
                    gmsh.model.mesh.getElementProperties(et)

                ip, wip =
                    gmsh.model.mesh.getIntegrationPoints(et, "Gauss" * string(2order + 1))

                # --- Lagrange shape functions (for proper interpolation of P)
                _, hfun, _ =
                    gmsh.model.mesh.getBasisFunctions(et, ip, "Lagrange")
                H = reshape(hfun, numNodes, :)

                # --- GradLagrange
                _, dfun, _ =
                    gmsh.model.mesh.getBasisFunctions(et, ip, "GradLagrange")
                ∇h = reshape(dfun, :, length(wip))

                for (e, elem) in enumerate(elemTags[it])
                    nodeTags = elemNodeTags[it][(e-1)*numNodes+1:e*numNodes]

                    jac, jacDet, _ =
                        gmsh.model.mesh.getJacobian(elem, ip)
                    Jac = reshape(jac, 3, :)

                    Pelem = Pmap[elem]
                    fe = zeros(pdim * numNodes)

                    for k in eachindex(wip)
                        invJ = inv(Jac[1:dim, 3k-2:3k])'
                        w = jacDet[k] * wip[k]

                        # --- interpolate P to Gauss point using shape functions
                        Pgp = zeros(dim, dim)
                        for a in 1:numNodes
                            Na = H[a, k]
                            Pgp .+= Na * reshape(Pelem[9a-8:9a, 1], dim, dim)
                        end

                        for a in 1:numNodes
                            ∇Na = invJ * ∇h[3a-2:3a-(3-dim), k]
                            for i in 1:dim
                                ia = (a - 1) * pdim + i
                                fe[ia] += dot(Pgp[i, :], ∇Na) * w
                            end
                        end
                    end

                    # scatter
                    for a in 1:numNodes, i in 1:dim
                        Ig = (nodeTags[a] - 1) * pdim + i
                        f[Ig] += fe[(a-1)*pdim+i]
                    end
                end
            end
        end
    end

    if pdim ≠ 3
        error("dim ≠ 3 is not implemented yet")
    end
    return VectorField([], reshape(f, :, 1), [0.0], [], 1, :v3D, problem)
end


In [ ]:
function externalTangentFollowerTL(
    problem::Problem;
    F::TensorField,
    traction_phName::AbstractString,
    t_spatial)

    gmsh.model.setCurrent(problem.name)

    dim = problem.dim
    pdim = problem.pdim
    non = problem.non
    dof = pdim * non

    Fe = nodesToElements(F)
    Fmap = Dict(zip(Fe.numElem, Fe.A))

    Id = zeros(dim, dim)
    for i in 1:dim
        Id[i, i] = 1
    end

    I = Int[]
    J = Int[]
    V = Float64[]

    for (edim, etag) in gmsh.model.getEntitiesForPhysicalName(traction_phName)
        elemTypes, elemTags, elemNodeTags =
            gmsh.model.mesh.getElements(edim, etag)

        for it in eachindex(elemTypes)
            et = elemTypes[it]
            _, _, order, numNodes, _, _ =
                gmsh.model.mesh.getElementProperties(et)

            ip, wip =
                gmsh.model.mesh.getIntegrationPoints(et, "Gauss" * string(2order + 1))

            # Lagrange shape functions
            _, hfun, _ =
                gmsh.model.mesh.getBasisFunctions(et, ip, "Lagrange")
            H = reshape(hfun, numNodes, :)

            # GradLagrange
            _, dfun, _ =
                gmsh.model.mesh.getBasisFunctions(et, ip, "GradLagrange")
            ∇h = reshape(dfun, :, length(wip))

            for (e, elem) in enumerate(elemTags[it])
                nodeTags = elemNodeTags[it][(e-1)*numNodes+1:e*numNodes]

                jac, jacDet, _ =
                    gmsh.model.mesh.getJacobian(elem, ip)
                Jac = reshape(jac, 3, :)

                Fnode = Fmap[elem]
                Ke = zeros(pdim * numNodes, pdim * numNodes)

                for k in eachindex(wip)
                    invJ = inv(Jac[1:dim, 3k-2:3k])'
                    w = jacDet[k] * wip[k]

                    # --- interpolate F to Gauss point using shape functions
                    Fgp = zeros(dim, dim)
                    for a in 1:numNodes
                        Na = H[a, k]
                        Fgp .+= Na * reshape(Fnode[9a-8:9a, 1], dim, dim)
                    end

                    Jgp = det(Fgp)
                    Finv = inv(Fgp)
                    FinvT = Finv'

                    if t_spatial isa Function
                        tgp = t_spatial(0.0, 0.0, 0.0)
                    else
                        tgp = collect(Float64, t_spatial)
                    end

                    for a in 1:numNodes, b in 1:numNodes
                        ∇Nb = invJ * ∇h[3b-2:3b-(3-dim), k]

                        # δF from δu_bj
                        for j in 1:dim
                            A = reshape(Finv[:, j], dim, 1) *
                                reshape(∇Nb, 1, dim)

                            dJFmT =
                                Jgp * FinvT * (tr(A) * Id - A')

                            dt0 = dJFmT * tgp

                            for i in 1:dim
                                ia = (a - 1) * pdim + i
                                jb = (b - 1) * pdim + j
                                Ke[ia, jb] += H[a, k] * dt0[i] * w
                            end
                        end
                    end
                end

                # scatter Ke
                for a in 1:pdim*numNodes, b in 1:pdim*numNodes
                    Ig = (nodeTags[div(a - 1, pdim)+1] - 1) * pdim + mod(a - 1, pdim) + 1
                    Jg = (nodeTags[div(b - 1, pdim)+1] - 1) * pdim + mod(b - 1, pdim) + 1
                    push!(I, Ig)
                    push!(J, Jg)
                    push!(V, Ke[a, b])
                end
            end
        end
    end

    K = sparse(I, J, V, dof, dof)
    dropzeros!(K)
    return SystemMatrix(K, problem)
end


In [ ]:
gmsh.initialize()
gmsh.open("cube1.geo")

mat = Material("cube")
prob = Problem([mat], type=:VectorField)

In [ ]:
F = TensorField(prob, "cube", [1 0 0; 0 1 0; 0 0 1]);

In [ ]:
K1 = poissonMatrixVector(prob, coefficient=F)
K1.A

In [ ]:
K2 = gradDivMatrixF(prob, coefficient=1, F=F)
K2.A

In [ ]:
K3 = poissonMatrixSymGradF(prob, coefficient=1, F=F)
K3.A

In [ ]:
fint = internalForceTL(prob, F)

In [ ]:
Fleft = nodesToElements(elementsToNodes(F), onPhysicalGroup="left")

Kext = externalTangentFollowerTL(prob; F=Fleft, traction_phName="left", t_spatial=[1, 0, 0])

In [ ]:
suppX = BoundaryCondition("left", ux=0, uy=0, uz=0)
suppY = BoundaryCondition("bottom", uy=0)
suppZ = BoundaryCondition("rear", uz=0)

load = BoundaryCondition("right", fx=10)

In [ ]:
f_ext = loadVector(prob, [load])

In [ ]:
u = vectorField(prob, "cube", [0, 0, 0])

In [ ]:
F = tensorField(prob, "cube", [1 0 0; 0 1 0; 0 0 1])

In [ ]:
I = tensorField(prob, "cube", [1 0 0; 0 1 0; 0 0 1])

In [ ]:
P = tensorField(prob, "cube", [0 0 0; 0 0 0; 0 0 0])

In [ ]:
μ = mat.μ
λ = mat.λ

## Innen indul az iteráció

In [ ]:
Kμ = poissonMatrixSymGradF(prob, coefficient=μ, F=F)
Kλ = gradDivMatrixF(prob, coefficient=λ, F=F)

Kint = Kμ + Kλ

In [ ]:
Fright = nodesToElements(elementsToNodes(F), onPhysicalGroup="right")

Kext = externalTangentFollowerTL(
    prob;
    F=Fright,
    traction_phName="right",
    t_spatial=[10.0, 0.0, 0.0]
)

In [ ]:
K = Kint #+ Kext

In [ ]:
f_int = internalForceTL(prob, P)

In [ ]:
Δu = solveField(
    K,
    f_ext - f_int;
    support=[suppX, suppY, suppZ]
)

norm(Δu.a[:, 1])

In [ ]:
u += Δu

In [ ]:
showDoFResults(u)
openPostProcessor()

In [ ]:
F = I + u ∘ ∇

In [ ]:
E = 0.5 * (F' * F - I)
S = λ * trace(E) * I + 2μ * E
P = F * S
#P = λ * trace(F - I) * I + 2μ * (F - I)

In [ ]:
#gmsh.finalize()